In [14]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
import org.apache.spark.sql.functions._

val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

def sc = spark.sparkContext
import spark.implicits._

import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark


import org.apache.spark.sql._


import org.apache.log4j.{Level, Logger}

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = 

In [19]:
var df = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("Data_02.csv").
        withColumn("date",to_date(col("timestamp"),"yyyy-MM-dd"))

df: DataFrame = [device_id: string, country_code: string ... 13 more fields]

In [36]:
// ----------------------------- Task 1 --------------------------------------------------------------
// Calculate how many users who used app in X day 
// a) returned the next day
// b) returned 7 day after

val df_1 = df.withColumn("date_plus_one", date_add(col("date"), 1)).
            select(col("device_id").as("DeviceId"), col("date").alias("Date"), col("date_plus_one").as("DatePlusOne")).
            join(df, 
                col("DeviceId") === col("device_id") && 
                col("DatePlusOne") === df.col("date")).
            select(col("DeviceId"), df.col("date"), col("DatePlusOne")).
            select(countDistinct("DeviceId").as("N of people who used app after 1 day")).
            show()


val df_2 = df.withColumn("date_plus_seven", date_add(col("date"), 7)).
            select(col("device_id").as("DeviceId"), col("date").alias("Date"), col("date_plus_seven").as("DatePlusSeven")).
            join(df, 
                col("DeviceId") === col("device_id") && 
                col("DatePlusSeven") === df.col("date")).
            select(col("DeviceId"), df.col("date"), col("DatePlusSeven")).
            select(countDistinct("DeviceId").as("N of people who used app after 7 days")).
            show()

+------------------------------------+
|N of people who used app after 1 day|
+------------------------------------+
|                                  69|
+------------------------------------+

+-------------------------------------+
|N of people who used app after 7 days|
+-------------------------------------+
|                                   49|
+-------------------------------------+



In [42]:
// ----------------------------- Task 2 --------------------------------------------------------------
// which tool has highest return rate (edit_text_apply, effect_apply, edit_item_open, edit_sticker_apply)

var df_2 = df.
    select(col("device_id").as("DeviceId"), col("session_id").as("SessionId"), col("event_type").as("EventType"), col("timestamp").as("Timestamp2")).
    filter($"EventType" === "edit_text_apply" || $"EventType" === "effect_apply" || 
           $"EventType" === "edit_item_open"  || $"EventType" === "edit_sticker_apply" )
df_2.join(df, col("DeviceId") === col("device_id") && col("EventType") === col("event_type") && col("Timestamp2") < col("timestamp")).
    select(col("DeviceId"), col("EventType").as("FirstEvent"), col("event_type").as("EventReapeated")).distinct().
    groupBy(col("FirstEvent")).agg(count("EventReapeated")).show(4, false)



+------------------+---------------------+
|FirstEvent        |count(EventReapeated)|
+------------------+---------------------+
|edit_text_apply   |21                   |
|edit_sticker_apply|10                   |
|effect_apply      |39                   |
|edit_item_open    |78                   |
+------------------+---------------------+



df_2: Dataset[Row] = [DeviceId: string, SessionId: string ... 2 more fields]

In [52]:
// ----------------------------- Task 3 --------------------------------------------------------------
// how many users did photo_like daily, in which day is it highest, if there is seasonality

var df_old = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("Data.csv").
    withColumn("date",to_date(col("timestamp"),"yyyy-MM-dd"))
var df_3 = df_old.select(col("date"), col("device_id"), col("event_type")).
    where(col("event_type") === "photo_like").
    select(col("date"), col("device_id")).distinct().
    groupBy(col("date")).
    agg(count("device_id")).as("photo_like_users").
    sort("date").
    show(30,false)




+----------+----------------+
|date      |count(device_id)|
+----------+----------------+
|2018-02-02|7               |
|2018-02-03|3               |
|2018-02-04|4               |
|2018-02-05|3               |
|2018-02-06|3               |
|2018-02-07|5               |
|2018-02-08|2               |
|2018-02-09|2               |
|2018-02-10|3               |
|2018-02-11|1               |
|2018-02-12|2               |
|2018-02-13|7               |
|2018-02-14|3               |
|2018-02-15|2               |
|2018-02-16|5               |
|2018-02-17|5               |
|2018-02-18|5               |
|2018-02-19|1               |
|2018-02-20|6               |
|2018-02-21|5               |
|2018-02-22|1               |
|2018-02-23|2               |
|2018-02-24|2               |
|2018-02-26|4               |
|2018-02-27|3               |
|2018-02-28|1               |
|2018-03-01|2               |
|2018-03-02|2               |
+----------+----------------+



df_old: DataFrame = [country_code: string, device_id: string ... 13 more fields]
df_3: Unit = ()

In [76]:
// ----------------------------- Task 4 --------------------------------------------------------------
// what is conversion rate that users who did effect_try then did effect apply

val df_4 = df.select(col("device_id").as("DeviceId"), 'event_type.as("EventType"), 'timestamp.as("Timestamp1"), 'date.as("Date1")).
    where(col("EventType") === "effect_try")
val eff_try_count = df_4.select(countDistinct("DeviceId")).take(1)(0)(0)
val eff_tr_app_count = df_4.join(df, 'DeviceId === 'device_id && 'event_type === "effect_apply" && 'Timestamp1 < 'timestamp && 'Date1 === 'date).
    select(countDistinct('DeviceId)).take(1)(0)(0)



df_4: Dataset[Row] = [DeviceId: string, EventType: string ... 2 more fields]
eff_try_count: Any = 56L
eff_tr_app_count: Any = 50L

In [111]:
// ----------------------------- Task 5 --------------------------------------------------------------
//find how many users did effect apply and what is descriptive statistics for it (outliers, avg, median)

df.select(countDistinct("device_id").as("total_number_of_users")).show(1)
val df_5 = df.where(col("event_type") === "effect_apply")
df_5.select(countDistinct("device_id").as("effect_apply_users")).show(1)
val count_ea = df_5.groupBy("device_id").
    agg(count("event_type").as("effect_apply")).sort("effect_apply")
count_ea.agg(max("effect_apply"), min("effect_apply"), avg("effect_apply")).show(5, false)

// this part calculates median
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
val median_rdd = spark.sparkContext.parallelize(Seq(count_ea.rdd.take(25).last))
val schema = new StructType(Array(new StructField("id", StringType, true), 
                            new StructField("median", LongType, true)))
val median_df = spark.createDataFrame(median_rdd, schema)
median_df.show(1, false)




+---------------------+
|total_number_of_users|
+---------------------+
|                   98|
+---------------------+

+------------------+
|effect_apply_users|
+------------------+
|                50|
+------------------+

+-----------------+-----------------+-----------------+
|max(effect_apply)|min(effect_apply)|avg(effect_apply)|
+-----------------+-----------------+-----------------+
|149              |1                |13.92            |
+-----------------+-----------------+-----------------+

+-----------------------------------------+------+
|id                                       |median|
+-----------------------------------------+------+
|andy-9674bf80-55df-4686-9547-118c87fb6607|4     |
+-----------------------------------------+------+



df_5: Dataset[Row] = [device_id: string, country_code: string ... 13 more fields]
count_ea: Dataset[Row] = [device_id: string, effect_apply: bigint]
import org.apache.spark.sql.Row

import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}

median_rdd: org.apache.spark.rdd.RDD[Row] = ParallelCollectionRDD[1027] at parallelize at cmd110.sc:10
schema: StructType = StructType(
  StructField("id", StringType, true, {}),
  StructField("median", LongType, true, {})
)
median_df: DataFrame = [id: string, median: bigint]

In [ ]:
// ----------------------------- Task 6 --------------------------------------------------------------
// what first 5 events users do in their session start  - 5 events in array and % of users did it



In [ ]:
// ----------------------------- Task 7 --------------------------------------------------------------
// What are the most important metrics when we want to describe one session of user in app

// 1. how long user stayed in the app (duration or difference of first and last timestamps)
// 2. usage of picsart tools
// 3. purcheses made by user
// 4. is it paid user or free subscribtion user